In [7]:
import camelot
import pandas as pd

In [ ]:
def merge_multipage_table_to_csv(
    pdf_path,
    pages="all",
    flavor="lattice",
    output_file="merged_table.csv",
    keep_header_from_first_page=True,
):
    """
    Parse a table that spans multiple pages and merge into a single CSV file

    Parameters:
    -----------
    pdf_path : str
        Path to the PDF file
    pages : str
        Pages to parse. Can be 'all', '1,2,3', or '1-3'
    flavor : str
        'lattice' for tables with lines, 'stream' for tables without lines
    output_file : str
        Name of the output CSV file
    keep_header_from_first_page : bool
        If True, uses header from first page only and skips headers on subsequent pages

    Returns:
    --------
    pandas.DataFrame: The merged table
    """

    try:
        # Read PDF tables
        print(f"Reading tables from {pdf_path}...")
        tables = camelot.read_pdf(
            pdf_path, pages=pages, flavor=flavor, table_areas=["28,813,567,27"]
        )

        if len(tables) == 0:
            print("No tables found in the PDF")
            return None

        print(f"Found {len(tables)} table(s) across pages")

        # Initialize merged dataframe
        merged_df = None

        for i, table in enumerate(tables):
            df = table.df
            print(f"  Page table {i+1}: Shape {df.shape}")

            if i == 0:
                # First table - keep as is
                merged_df = df
                if keep_header_from_first_page:
                    # Store the header for comparison
                    header_row = df.iloc[0].astype(str).str.strip()
            else:
                # Subsequent tables
                if keep_header_from_first_page:
                    # Check if first row matches the header pattern
                    current_first_row = df.iloc[0].astype(str).str.strip()

                    # If first row looks like a header (matches the original header), skip it
                    if current_first_row.equals(header_row):
                        df = df.iloc[1:]  # Skip the header row
                        print(f"    Skipped duplicate header row")

                # Append to merged dataframe
                merged_df = pd.concat([merged_df, df], ignore_index=True)

        # Clean up the merged dataframe
        # Option 1: If first row contains headers, set it as column names
        if keep_header_from_first_page and not merged_df.empty:
            # Check if the first row looks like headers (you can customize this logic)
            first_row = merged_df.iloc[0].astype(str)
            if all(first_row.str.len() > 0):  # Basic check for non-empty headers
                merged_df.columns = first_row
                merged_df = merged_df[1:].reset_index(drop=True)
                print("\nSet first row as column headers")

        # Remove any completely empty rows
        merged_df = merged_df.dropna(how="all").reset_index(drop=True)

        # Save to CSV
        merged_df.to_csv(output_file, index=False)
        print(f"\nMerged table saved to: {output_file}")
        print(f"Final table shape: {merged_df.shape}")

        # Display preview
        print("\nPreview of merged table:")
        print(merged_df.head(10))

        return merged_df, tables

    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None


def merge_with_custom_header_handling(
    pdf_path,
    pages="all",
    flavor="lattice",
    output_file="merged_table.csv",
    header_rows_to_skip=1,
):
    """
    Alternative method: Skip a specific number of rows (headers) from each page except the first

    Parameters:
    -----------
    pdf_path : str
        Path to the PDF file
    pages : str
        Pages to parse
    flavor : str
        'lattice' or 'stream'
    output_file : str
        Output CSV filename
    header_rows_to_skip : int
        Number of header rows to skip on pages after the first
    """

    try:
        print(f"Reading tables from {pdf_path}...")
        tables = camelot.read_pdf(pdf_path, pages=pages, flavor=flavor)

        if len(tables) == 0:
            print("No tables found")
            return None

        print(f"Found {len(tables)} table(s)")

        # Collect all dataframes
        dfs = []
        for i, table in enumerate(tables):
            df = table.df

            if i == 0:
                # Keep the first page as is
                dfs.append(df)
                print(f"  Page {i+1}: Keeping all {len(df)} rows")
            else:
                # Skip header rows on subsequent pages
                df_trimmed = df.iloc[header_rows_to_skip:]
                dfs.append(df_trimmed)
                print(
                    f"  Page {i+1}: Skipping {header_rows_to_skip} header row(s), keeping {len(df_trimmed)} rows"
                )

        # Merge all dataframes
        merged_df = pd.concat(dfs, ignore_index=True)

        # Clean up
        merged_df = merged_df.dropna(how="all").reset_index(drop=True)

        # Save to CSV
        merged_df.to_csv(output_file, index=False)
        print(f"\nSaved merged table to: {output_file}")
        print(f"Total rows: {len(merged_df)}")

        return merged_df

    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
if __name__ == "__main__":
    df, tables = merge_multipage_table_to_csv(
        pdf_path="oficjalny_spis_pna_2025.pdf",
        pages="3-20",
        flavor="stream",
        output_file="merged_output.csv",
        keep_header_from_first_page=True,
    )

Reading tables from oficjalny_spis_pna_2025.pdf...
Found 18 table(s) across pages
  Page table 1: Shape (75, 6)
  Page table 2: Shape (83, 6)
  Page table 3: Shape (83, 6)
  Page table 4: Shape (83, 6)
  Page table 5: Shape (83, 7)
  Page table 6: Shape (82, 6)
  Page table 7: Shape (83, 6)
  Page table 8: Shape (83, 6)
  Page table 9: Shape (83, 6)
  Page table 10: Shape (83, 7)
  Page table 11: Shape (83, 6)
  Page table 12: Shape (83, 6)
  Page table 13: Shape (83, 6)
  Page table 14: Shape (83, 7)
  Page table 15: Shape (83, 7)
  Page table 16: Shape (83, 6)
  Page table 17: Shape (83, 6)
  Page table 18: Shape (83, 7)

Merged table saved to: merged_output.csv
Final table shape: (1485, 7)

Preview of merged table:
        0                        1       2          3             4  \
0          PNA miejscowości i ulic                                    
1     PNA       Miejscowość\nUlica  Numery      Gmina        Powiat   
2  83-440                 Abisynia             Karsin    ko